# Análise sobre os Preços dos Combustíveis no Brasil

Aluno: Rafael Pereira Cândido

Matrícula: 2221134

Curso: Ciência da Computação

Esse trabalho tem o propósito de analizar os preços dos combustíveis no Brasil do período de 2003 até 2023.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [9]:
spark = SparkSession.builder \
        .appName("Processamento de Chunks") \
        .getOrCreate()

In [10]:
def handle_data(path):
  df = spark.read.csv(path, sep=';', inferSchema=True, header=True)

  # Sanitize data
  df = df.withColumn(
      "Data da Coleta",
      f.to_date(f.col("Data da Coleta").cast(StringType()), 'dd/MM/yyyy')
      )\
      .withColumn('Valor de Venda', f.regexp_replace('Valor de Venda', ',', '.'))\
      .withColumn('Valor de Venda', f.col('Valor de Venda').cast(DoubleType()))\
      .withColumn('Valor de Compra', f.regexp_replace('Valor de Compra', ',', '.'))\
      .withColumn('Valor de Compra', f.col('Valor de Compra').cast(DoubleType()))\
      .fillna({'Complemento': 'S/C'})

  return df

In [11]:
base_path = '/home/jovyan/data/'

In [13]:
path = base_path+'ca-2022-01.csv'
df = handle_data(path)

In [7]:
df.show(5, False)

+--------------+--------------+----------+-------------------------+-------------------+----------------+----------+-----------+------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+
|Regiao - Sigla|Estado - Sigla|Municipio |Revenda                  |CNPJ da Revenda    |Nome da Rua     |Numero Rua|Complemento|Bairro|Cep      |Produto           |Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira     |
+--------------+--------------+----------+-------------------------+-------------------+----------------+----------+-----------+------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+
|N             |AC            |RIO BRANCO|AUTO POSTO AMAPA - EIRELI| 00.529.581/0001-53|VIA CHICO MENDES|3570      |NULL       |AREAL |69906-119|GASOLINA          |03/01/2022    |6,99          |NULL           |R$ / litro       |VIBRA ENERGIA|
|N             |AC          

In [16]:
df.printSchema()
print(df.count())

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = false)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: date (nullable = true)
 |-- Valor de Venda: double (nullable = true)
 |-- Valor de Compra: double (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)

529424


In [17]:
df_gasolina = df.filter(df.Produto == 'GASOLINA')

In [18]:
df_gasolina.show(5, False)

+--------------+--------------+----------+-------------------------+-------------------+---------------------------------------+----------+-----------+----------+---------+--------+--------------+--------------+---------------+-----------------+-------------+
|Regiao - Sigla|Estado - Sigla|Municipio |Revenda                  |CNPJ da Revenda    |Nome da Rua                            |Numero Rua|Complemento|Bairro    |Cep      |Produto |Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira     |
+--------------+--------------+----------+-------------------------+-------------------+---------------------------------------+----------+-----------+----------+---------+--------+--------------+--------------+---------------+-----------------+-------------+
|N             |AC            |RIO BRANCO|AUTO POSTO AMAPA - EIRELI| 00.529.581/0001-53|VIA CHICO MENDES                       |3570      |S/C        |AREAL     |69906-119|GASOLINA|2022-01-03    |6.99          |NULL     

In [19]:
# Média do preço da gasolina de todos os dados

media_gasolina_todos = df_gasolina.agg(f.avg('Valor de Venda').alias('media_valor_venda_gasolina'))
media_gasolina_todos.show()

+--------------------------+
|media_valor_venda_gasolina|
+--------------------------+
|          7.02321269256594|
+--------------------------+



In [20]:
# Média do preço da gasolina no estado do DF
media_gasolina_df = df_gasolina.filter(df_gasolina['Estado - Sigla'] == 'DF') \
    .agg(f.avg('Valor de Venda').alias('media_valor_venda_gasolina_df'))
media_gasolina_df.show()

+-----------------------------+
|media_valor_venda_gasolina_df|
+-----------------------------+
|            7.221960461285014|
+-----------------------------+



In [21]:
media_gasolina_por_estado = df_gasolina.groupBy('Estado - Sigla')\
    .agg(f.avg('Valor de Venda').alias('media_valor_venda_gasolina'))\
    .orderBy('Estado - Sigla')
media_gasolina_por_estado.show(50, False)

+--------------+--------------------------+
|Estado - Sigla|media_valor_venda_gasolina|
+--------------+--------------------------+
|AC            |7.412036108324978         |
|AL            |7.093333333333335         |
|AM            |7.220469226081652         |
|AP            |6.224166666666665         |
|BA            |7.433811199061843         |
|CE            |7.231936131753342         |
|DF            |7.221960461285014         |
|ES            |7.1757544378698235        |
|GO            |7.288086649676514         |
|MA            |7.064512987012989         |
|MG            |7.348636777423767         |
|MS            |7.003534617386782         |
|MT            |6.7917960634226135        |
|PA            |7.269841526520049         |
|PB            |6.901242866201656         |
|PE            |7.007469177314493         |
|PI            |7.610396158463386         |
|PR            |6.934694251190036         |
|RJ            |7.616037356661521         |
|RN            |7.38585219707057